# Draw Prime Climb Board by Bokeh

You can output the figure to HTML, and print it to get a free Prime Climb board.

https://mathforlove.com/games/prime-climb/

In [ ]:
import narwhals
import numpy as np
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20_20
from bokeh.core.properties import value
from bokeh.io import export_png
from functools import lru_cache
from shapely import geometry
output_notebook()

In [ ]:
Colors = {
    1: "#cccccc",
    2: Category20_20[2],
    3: Category20_20[4],
    5: Category20_20[0],
    7: Category20_20[8],
    0: Category20_20[6],
}

In [ ]:
@lru_cache()
def get_primes(n):
    numbers = list(range(2, n+1))
    primes = []
    while numbers:
        primes.append(numbers[0])
        numbers = [x for x in numbers if x % primes[-1] != 0]
    return primes

def get_factors(x):
    if x == 1:
        yield 1
    else:
        primes = get_primes(101)
        for p in primes:
            while True:
                if x % p == 0:
                    yield p
                    x = x // p
                else:
                    break

In [ ]:
def extend_dict(d, data):
    for key, val in data.items():
        d[key].extend(val)

        
class PrimeClimb:
    def __init__(self, data_width, image_width, inner_radius=0.5, outer_radius=0.8):
        self.scale = image_width / data_width
        self.data_width = data_width
        self.image_width = image_width
        self.font_size = int(round(inner_radius * 0.8 * self.scale))
        self.line_width = inner_radius * 0.08 * self.scale
        self.inner_radius = inner_radius
        self.outer_radius = outer_radius
        
        self.annular_wedges = dict(
            x=[],
            y=[],
            start_angle=[],
            end_angle=[],
            fill_color=[],
        )
        
        self.annulus = dict(
            x=[],
            y=[],
            fill_color=[],
        )
        
        self.texts = dict(
            x=[],
            y=[],
            text=[],
            text_font_size=[],
            color=[],
        )
        
        self.circles = dict(
            x=[],
            y=[]
        )
        
        self.fig = figure(width=self.image_width, height=self.image_width, 
                          match_aspect=True, output_backend="canvas", toolbar_location=None)
        self.fig.background_fill_color  = "#333333"
        self.fig.xaxis.visible = False
        self.fig.yaxis.visible = False
        self.fig.xgrid.visible = False
        self.fig.ygrid.visible = False
        self.fig.x_range.update(start=0, end=self.data_width)
        self.fig.y_range.update(start=self.data_width, end=0)
        
    def make_number(self, n, max_number):
        x, y = self.get_number_location(n, max_number)            
        factors = list(get_factors(n))
        count = len(factors)
        colors = [Colors[f] if f in Colors else Colors[0] for f in factors]
        if len(factors) == 1:
            data = dict(x=[x], y=[y], fill_color=colors)
            extend_dict(self.annulus, data)
        else:
            angle = np.pi * 2 / len(factors)
            start_angle = np.arange(0, np.pi*2, angle)
            end_angle = start_angle + angle
            data = dict(
                x=[x] * count,
                y=[y] * count,
                start_angle=start_angle.tolist(),
                end_angle=end_angle.tolist(),
                fill_color=colors
            )
            extend_dict(self.annular_wedges, data)
            
        data = dict(x=[x], y=[y])
        extend_dict(self.circles, data)
        
        data = dict(x=[x], y=[y], text=[str(n)], text_font_size=["{}px".format(self.font_size)], color=["black"])
        if len(factors) >= 2:
            for f, sa, ea in zip(factors, start_angle, end_angle):
                if f > 7:
                    angle = 0.5 * (sa + ea)
                    r = 0.5 * (self.inner_radius + self.outer_radius)
                    dx = r * np.cos(angle)
                    dy = r * np.sin(angle)
                    data["x"].append(x + dx)
                    data["y"].append(y - dy)
                    data["text"].append(str(f))
                    data["color"].append("white")
                    data["text_font_size"].append("{}px".format(self.font_size // 2))
        extend_dict(self.texts, data)

    def build(self, max_number):
        for i in range(max_number):
            n = i + 1
            self.make_number(n, max_number)

    def draw_annular_wedges(self):
        source = ColumnDataSource(data=self.annular_wedges)
        self.fig.annular_wedge(x="x", y="y", 
                          inner_radius=self.inner_radius, outer_radius=self.outer_radius,
                          start_angle="start_angle", end_angle="end_angle", 
                          fill_color="fill_color",
                          alpha=1.0,
                          source=source, 
                          line_width=self.line_width, line_color="white")

    def draw_annulus(self):
        source = ColumnDataSource(data=self.annulus)
        self.fig.annulus(x="x", y="y",
                  inner_radius=self.inner_radius, outer_radius=self.outer_radius,
                  fill_color="fill_color",
                  source=source,
                  alpha=1.0,
                  line_width=self.line_width, line_color="white")
        
    def draw_circles(self):
        source = ColumnDataSource(data=self.circles)
        self.fig.circle(x="x", y="y", radius=self.inner_radius, fill_color="white", source=source)
        
    def draw_text(self):
        source = ColumnDataSource(data=self.texts)
        self.fig.text(x="x", y="y", text="text", text_color="color", source=source,
                 text_baseline="middle", text_align="center", y_offset=0, 
                 text_font_size="text_font_size",
                 text_font=value('cambria'),
                 text_font_style="bold")

    def draw(self):
        self.draw_circles()
        self.draw_annular_wedges()
        self.draw_annulus()
        self.draw_text()   
        
    def show(self):
        show(self.fig)
        
class PrimeClimbGrid(PrimeClimb):
    def get_number_location(self, n, max_number):
        x = ((n-1) % 10) * 2 + 1 + (self.data_width - 20) * 0.5
        y = ((n-1) // 10) * 2 + 1 + (self.data_width - 20) * 0.5
        return x, y

class PrimeClimbSpin(PrimeClimb):
    def __init__(self, r_min, r_max, *args, **kw):
        super().__init__(*args, **kw)
        self.n = n = 1000
        a = np.linspace(0, 9*np.pi, n) + np.pi + np.pi/2
        r = np.linspace(r_min, r_max, n)
        x = -r * np.cos(a) + self.data_width * 0.5
        y = r * np.sin(a) + self.data_width * 0.5
        self.x = x
        self.y = y
        self.length = np.r_[0, np.cumsum(np.hypot(np.diff(self.x), np.diff(self.y)))]
        self.length /= self.length[-1]
        s = geometry.LineString(list(zip(x, y)))
        x, y = np.asarray(s.buffer(0.2).exterior.coords).T
        self.x_spin = x
        self.y_spin = y
        
    def get_number_location(self, n, max_number):
        p = (n - 1) / (max_number - 1)
        loc = np.searchsorted(self.length, 1 - p)
        return self.x[loc], self.y[loc]
        
    def draw_spin(self):
        self.fig.patch(x=self.x_spin, y=self.y_spin, color="#888888")
        
    def draw(self):
        self.draw_spin()
        super().draw()

In [ ]:
pc = PrimeClimbGrid(data_width=25, image_width=800)
pc.build(100)
pc.draw()
pc.show()

In [ ]:
pc = PrimeClimbSpin(data_width=33, image_width=800, r_min=0.5, r_max=15)
pc.build(101)
pc.draw()
pc.show()